<a href="https://colab.research.google.com/github/abhishek203/E-Abhishek/blob/master/chatbot_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd 
import tensorflow as tf
import re
import time
import io
import os
from keras.preprocessing.sequence import pad_sequences
from keras.utils import plot_model
from keras.models import Model
from keras.layers import Input, LSTM, Dense, Embedding, Bidirectional, concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
import time
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split



Using TensorFlow backend.


In [2]:
path_to_zip = tf.keras.utils.get_file(
    'cornell movie-dialogs corpus.zip', origin='http://www.cs.cornell.edu/~cristian/data/cornell_movie_dialogs_corpus.zip',
    extract=True)

path_to_lines = os.path.dirname(path_to_zip)+"/cornell movie-dialogs corpus/movie_lines.txt"
path_to_conv = os.path.dirname(path_to_zip)+"/cornell movie-dialogs corpus/movie_conversations.txt"

9920512/9916637 [==============================] - 3s 0us/step


In [0]:
lines = io.open(path_to_lines,encoding = 'utf-8',errors='ignore').read().strip().split('\n')
conv_lines = io.open(path_to_conv,errors = 'ignore').read().strip().split('\n')

In [0]:
id2line ={}
for line in lines:
  _line = line.split(' +++$+++ ')
  if len(_line) ==5:
      id2line[_line[0]] = _line[4]


In [0]:
convs = []
for line in conv_lines:
    _line = line.split(' +++$+++ ')[-1][1:-1].replace(r"'","").replace(" ","")
    convs.append(_line.split(','))

In [0]:
#question and answer
question = []
answer = []
for k in convs:
  for l in range(len(k)-1):
    question.append(id2line[k[l]])
    answer.append(id2line[k[l+1]])

In [0]:
def clean_text(text):
    '''Clean text by removing unnecessary characters and altering the format of words.'''

    text = text.lower()
    
    text = re.sub(r"i'm", "i am", text)
    text = re.sub(r"he's", "he is", text)
    text = re.sub(r"she's", "she is", text)
    text = re.sub(r"it's", "it is", text)
    text = re.sub(r"that's", "that is", text)
    text = re.sub(r"what's", "that is", text)
    text = re.sub(r"where's", "where is", text)
    text = re.sub(r"how's", "how is", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"can't", "cannot", text)
    text = re.sub(r"n't", " not", text)
    text = re.sub(r"n'", "ng", text)
    text = re.sub(r"'bout", "about", text)
    text = re.sub(r"'til", "until", text)
    text = re.sub(r"[-()\"#/@;:<>{}`+=~|.!?,]", "", text)
    text = " ".join(text.split())
    return text

In [0]:
clean_questions = []
for aquestion in question:
    clean_questions.append(clean_text(aquestion))
    
clean_answers = []    
for aanswer in answer:
    clean_answers.append(clean_text(aanswer))

In [0]:
min_length = 2
max_length =20
l = 0
short_question_temp = []
short_ans_temp = []
for aquestion in clean_questions:
  if len(aquestion)>=min_length and len(aquestion)<=max_length:
    short_question_temp.append(aquestion)
    short_ans_temp.append(clean_answers[l])
  l+=1

In [0]:
short_ans = []
short_question = []
m = 0
for aanswer in short_ans_temp:
  if len(aanswer)>=min_length and len(aanswer)<=max_length:
    short_ans.append(aanswer)
    short_question.append(short_question_temp[m])
  m+=1

In [0]:
for i in range(len(short_question)):
  short_question[i]='<start> '+ short_question[i] +' <end>'
for i in range(len(short_ans)):
  short_ans[i] = '<start> '+ short_ans[i] +' <end>'


In [12]:
short_ans[90]

'<start> it is okay <end>'

In [0]:
#tokenizer
tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='',num_words=1000,oov_token='<oov>')

tokenizer.fit_on_texts(short_question)
question_seq = tokenizer.texts_to_sequences(short_question)
question_seq = tf.keras.preprocessing.sequence.pad_sequences(question_seq,padding='post')
tokenizer.fit_on_texts(short_ans)
ans_seq = tokenizer.texts_to_sequences(short_ans)
ans_seq = tf.keras.preprocessing.sequence.pad_sequences(ans_seq, padding = 'post')

In [0]:
len_input_seq = len(question_seq[0])
len_output_seq = len(ans_seq[0])


BUFFER_SIZE = len(question_seq)
BATCH_SIZE = 64
steps_per_epoch = len(question_seq)//BATCH_SIZE
embedding_dim = 256
units = 1024
vocab_inp_size = len(tokenizer.word_index)+1
vocab_tar_size = len(tokenizer.word_index)+1

dataset = tf.data.Dataset.from_tensor_slices((question_seq, ans_seq)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)


In [15]:
len(question_seq)

22928

In [16]:
example_input_batch, example_target_batch = next(iter(dataset))
example_input_batch.shape, example_target_batch.shape

(TensorShape([64, 9]), TensorShape([64, 9]))

In [17]:
tokenizer.word_index['<start>']

2

In [0]:
class Encoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
    super(Encoder, self).__init__()
    self.batch_sz = batch_sz
    self.enc_units = enc_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.enc_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')

  def call(self, x):
    x = self.embedding(x)
    output, state_h = self.gru(x)
    return output, state_h


In [0]:
class BahdanauAttention(tf.keras.layers.Layer):
  def __init__(self, units):
    super(BahdanauAttention, self).__init__()
    self.W1 = tf.keras.layers.Dense(units)
    self.W2 = tf.keras.layers.Dense(units)
    self.V = tf.keras.layers.Dense(1)

  def call(self, query, values):
    query_with_time_axis = tf.expand_dims(query, 1)

    score = self.V(tf.nn.tanh(
        self.W1(query_with_time_axis) + self.W2(values)))

    attention_weights = tf.nn.softmax(score, axis=1)

    context_vector = attention_weights * values
    context_vector = tf.reduce_sum(context_vector, axis=1)

    return context_vector, attention_weights

In [0]:
class Decoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
    super(Decoder, self).__init__()
    self.batch_sz = batch_sz
    self.dec_units = dec_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.dec_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')
    self.fc = tf.keras.layers.Dense(vocab_size)

    self.attention = BahdanauAttention(self.dec_units)

  def call(self, x, hidden, enc_output):
    context_vector, attention_weights = self.attention(hidden, enc_output)

    x = self.embedding(x)

    x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)

    output,state= self.gru(x)

    output = tf.reshape(output, (-1, output.shape[2]))

    x = self.fc(output)

    return x, state, attention_weights

In [0]:
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)
decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE)

In [0]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_mean(loss_)

In [0]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)


In [0]:
@tf.function
def train_step(inp, targ):
  loss = 0

  with tf.GradientTape() as tape:
    enc_output, enc_hidden = encoder(inp)

    dec_hidden = enc_hidden

    dec_input = tf.expand_dims([tokenizer.word_index['<start>']] * BATCH_SIZE, 1)

    for t in range(1, targ.shape[1]):
      predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)

      loss += loss_function(targ[:, t], predictions)

      dec_input = tf.expand_dims(targ[:, t], 1)

  batch_loss = (loss / int(targ.shape[1]))

  variables = encoder.trainable_variables + decoder.trainable_variables

  gradients = tape.gradient(loss, variables)

  optimizer.apply_gradients(zip(gradients, variables))

  return batch_loss

In [39]:
EPOCHS = 10

for epoch in range(EPOCHS):
  start = time.time()

  total_loss = 0

  for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
    batch_loss = train_step(inp, targ)
    total_loss += batch_loss

    if batch % 100 == 0:
      print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
                                                   batch,
                                                   batch_loss.numpy()))
  # saving (checkpoint) the model every 2 epochs
  if (epoch + 1) % 2 == 0:
    checkpoint.save(file_prefix = checkpoint_prefix)

  print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                      total_loss / steps_per_epoch))
  print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

Epoch 1 Batch 0 Loss 0.8925
Epoch 1 Batch 100 Loss 0.8481
Epoch 1 Batch 200 Loss 0.8583
Epoch 1 Batch 300 Loss 0.8842
Epoch 1 Loss 0.8758
Time taken for 1 epoch 20.924778938293457 sec

Epoch 2 Batch 0 Loss 0.6877
Epoch 2 Batch 100 Loss 0.8238
Epoch 2 Batch 200 Loss 0.6646
Epoch 2 Batch 300 Loss 0.7780
Epoch 2 Loss 0.7780
Time taken for 1 epoch 21.592979431152344 sec

Epoch 3 Batch 0 Loss 0.6310
Epoch 3 Batch 100 Loss 0.6420
Epoch 3 Batch 200 Loss 0.5540
Epoch 3 Batch 300 Loss 0.7144
Epoch 3 Loss 0.6740
Time taken for 1 epoch 20.96075677871704 sec

Epoch 4 Batch 0 Loss 0.5330
Epoch 4 Batch 100 Loss 0.5512
Epoch 4 Batch 200 Loss 0.6228
Epoch 4 Batch 300 Loss 0.5118
Epoch 4 Loss 0.5704
Time taken for 1 epoch 21.382102489471436 sec

Epoch 5 Batch 0 Loss 0.4796
Epoch 5 Batch 100 Loss 0.4983
Epoch 5 Batch 200 Loss 0.5249
Epoch 5 Batch 300 Loss 0.4850
Epoch 5 Loss 0.4842
Time taken for 1 epoch 20.968665599822998 sec

Epoch 6 Batch 0 Loss 0.3439
Epoch 6 Batch 100 Loss 0.4534
Epoch 6 Batch 200 

In [0]:
def evaluate(sentence):

  sentence = clean_text(sentence)
  sentence = '<start> ' + sentence + ' <end>'
  tokenizer.fit_on_texts(sentence)

  inputs = [tokenizer.word_index[i] for i in sentence.split(' ')]
  inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],
                                                         maxlen=len_input_seq,
                                                         padding='post')
  inputs = tf.convert_to_tensor(inputs)

  result = ''

  enc_out, enc_hidden = encoder(inputs)

  dec_hidden = enc_hidden
  dec_input = tf.expand_dims([tokenizer.word_index['<start>']], 0)

  for t in range(len_output_seq):
    predictions, dec_hidden, attention_weights = decoder(dec_input,
                                                         dec_hidden,
                                                         enc_out)

    predicted_id = tf.argmax(predictions[0]).numpy()

    result += tokenizer.index_word[predicted_id] + ' '

    if tokenizer.index_word[predicted_id] == '<end>':
      return result, sentence

    dec_input = tf.expand_dims([predicted_id], 0)

  return result, sentence

In [0]:
def speak(sentence):
  result, sentence = evaluate(sentence)

  print('Input: %s' % (sentence))
  print('Answer: {}'.format(result))


In [42]:
# restoring the latest checkpoint in checkpoint_dir
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

In [43]:
speak('the weather is very hot today')

Input: <start> the weather is very hot today <end>
Answer: where <end> 


In [44]:
speak('why are you')

Input: <start> why are you <end>
Answer: you did <end> 


In [45]:
speak('where is he')

Input: <start> where is he <end>
Answer: i do not know <end> 


In [46]:
speak('where is she')

Input: <start> where is she <end>
Answer: oh i do not <end> 


In [47]:
speak('you want to go out')

Input: <start> you want to go out <end>
Answer: you got a <oov> <end> 


In [48]:
speak('do you know someone from us')

Input: <start> do you know someone from us <end>
Answer: what <end> 


In [49]:
speak('how are you')

Input: <start> how are you <end>
Answer: no <end> 


In [50]:
speak('do you know english')

Input: <start> do you know english <end>
Answer: what <end> 


In [51]:
speak('how were you made')

Input: <start> how were you made <end>
Answer: if i do <end> 


In [52]:
speak('who made you')

Input: <start> who made you <end>
Answer: yeah <end> 
